### Building a RAG System with LangChain and FAISS 
Introduction to RAG (Retrieval-Augmented Generation)
RAG combines the power of retrieval systems with generative AI models. Instead of relying solely on the model's training data, RAG:

1. Retrieves relevant documents from a knowledge base
2. Uses these documents as context for the LLM
3. Generates responses based on both the retrieved context and the model's knowledge

### FAISS 
https://github.com/facebookresearch/faiss

FAISS is a library for efficient similarity search and clustering of dense vectors.

Key advantages:
1. Extremely fast similarity search
2. Memory efficient
3. Supports GPU acceleration
4. Can handle millions of vectors

How it works:
- Indexes vectors for fast nearest neighbor search
- Returns most similar vectors based on distance metrics


In [ ]:
## load libraries
import os
from dotenv import load_dotenv
import langchain_community
import numpy as np

# LangChain core imports
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader

from langchain_classic.prompts import ChatPromptTemplate
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage,AIMessage
load_dotenv()

/home/aniruddha/Projects/RAG/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

### Data Ingestion And Processing


In [3]:
sample_documents = [
    Document(
        page_content="""
        Artificial Intelligence (AI) is the simulation of human intelligence in machines.
        These systems are designed to think like humans and mimic their actions.
        AI can be categorized into narrow AI and general AI.
        """,
        metadata={"source": "AI Introduction", "page": 1, "topic": "AI"}
    ),
    Document(
        page_content="""
        Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised, unsupervised, and reinforcement learning.
        """,
        metadata={"source": "ML Basics", "page": 1, "topic": "ML"}
    ),
    Document(
        page_content="""
        Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses multiple layers to progressively extract higher-level features from raw input.
        Deep learning has revolutionized computer vision, NLP, and speech recognition.
        """,
        metadata={"source": "Deep Learning", "page": 1, "topic": "DL"}
    ),
    Document(
        page_content="""
        Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
        It combines computational linguistics with machine learning and deep learning models.
        Applications include chatbots, translation, sentiment analysis, and text summarization.
        """,
        metadata={"source": "NLP Overview", "page": 1, "topic": "NLP"}
    )
]

print(sample_documents)

[Document(metadata={'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}, page_content='\n        Artificial Intelligence (AI) is the simulation of human intelligence in machines.\n        These systems are designed to think like humans and mimic their actions.\n        AI can be categorized into narrow AI and general AI.\n        '), Document(metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='\n        Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.\n        '), Document(metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='\n        Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolu

In [4]:
## text splitting
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", " ", ""],
    length_function=len
)

## split the documents into chunks
docs = text_splitter.split_documents(sample_documents)
docs

[Document(metadata={'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}, page_content='Artificial Intelligence (AI) is the simulation of human intelligence in machines.\n        These systems are designed to think like humans and mimic their actions.\n        AI can be categorized into narrow AI and general AI.'),
 Document(metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.'),
 Document(metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolutionized computer vision, NLP, and speech recogn

In [5]:
print(docs[0])
print(docs[1])

page_content='Artificial Intelligence (AI) is the simulation of human intelligence in machines.
        These systems are designed to think like humans and mimic their actions.
        AI can be categorized into narrow AI and general AI.' metadata={'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}
page_content='Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised, unsupervised, and reinforcement learning.' metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}


In [6]:
print(f"Created {len(docs)} chunks from {len(sample_documents)} documents")
print("\nExample chunk:")
print(f"Content: {docs[0].page_content}")
print(f"Metadata: {docs[0].metadata}")

Created 4 chunks from 4 documents

Example chunk:
Content: Artificial Intelligence (AI) is the simulation of human intelligence in machines.
        These systems are designed to think like humans and mimic their actions.
        AI can be categorized into narrow AI and general AI.
Metadata: {'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}


In [7]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [8]:
# Initialize OpenAI embeddings with the latest model
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    dimensions=1536
)
## Example: create a embedding for a single text
sample_text="What is machine learning"
sample_embedding = embeddings.embed_query(sample_text)
print(f"Embedding vector for the text '{sample_text}':\n{sample_embedding[:10]}")

Embedding vector for the text 'What is machine learning':
[-0.0059221177361905575, -0.005889697000384331, 0.0005751235294155777, -0.033544257283210754, 0.021192103624343872, 0.02213229238986969, -0.0007868689717724919, 0.00929923728108406, -0.022564563900232315, 0.037823744118213654]


In [9]:
texts=["AI","MAchine learning","Deep Learning","Neural Network"]
batch_embeddings = embeddings.embed_documents(texts)
for i, text in enumerate(texts):
    print(f"Embedding vector for the text '{text}':\n{batch_embeddings[i][:10]}")

Embedding vector for the text 'AI':
[-0.008208601735532284, -0.024612585082650185, 0.002946041990071535, 0.025167757645249367, 0.006533174309879541, -0.02826085314154625, -0.0050229765474796295, 0.020977536216378212, -0.036879222840070724, 0.012868072837591171]
Embedding vector for the text 'MAchine learning':
[-0.018237262964248657, 0.010781585238873959, 0.017500899732112885, -0.03527180105447769, 0.037456344813108444, 0.02609180472791195, 0.01585635542869568, 0.008449768647551537, -0.010910448618233204, 0.034903619438409805]
Embedding vector for the text 'Deep Learning':
[0.018034912645816803, -0.03604638949036598, -0.011003757826983929, -8.271678234450519e-05, 0.029038671404123306, 0.004886652808636427, 0.006210853811353445, 0.03053865022957325, -0.04973370581865311, 0.043897844851017]
Embedding vector for the text 'Neural Network':
[-0.02119721658527851, -0.005830408073961735, -0.007417852059006691, -0.0018001788994297385, 0.039248887449502945, -0.020774681121110916, 0.039366260170

In [10]:
### Compare Embedding using cosine similarity
def compare_embeddings(text1: str, text2: str):
    """Compare semantic simialrity of 2 texts usign embeddings"""
    
    embedding1 = np.array(embeddings.embed_query(text1))
    embedding2 = np.array(embeddings.embed_query(text2))
   
    ## Calculate the simialrity score
    simpilarity_score = np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))
    return simpilarity_score

In [11]:
# Test semantic similarity
print("\nSemantic Similarity Examples:")
print(f"'AI' vs 'Artificial Intelligence': {compare_embeddings('AI', 'Artificial Intelligence'):.3f}")


Semantic Similarity Examples:
'AI' vs 'Artificial Intelligence': 0.563


In [12]:
print(f"'Artificial Intelligence' vs 'DOG': {compare_embeddings('Artificial Intelligence', 'DOG'):.3f}")

'Artificial Intelligence' vs 'DOG': 0.149


### Create FAISS Vector Store

In [13]:
vectorstores = FAISS.from_documents(
    docs, embeddings
)

print(f"FAISS vector store created with the document chunks: {vectorstores.index.ntotal}")

FAISS vector store created with the document chunks: 4


In [14]:
vectorstores

In [15]:
vectorstores.save_local("faiss_vectorstore")

In [16]:
## load vector store
loaded_vectorstore = FAISS.load_local(
    "faiss_vectorstore", 
    embeddings,
    allow_dangerous_deserialization=True
)
print(f"Loaded vector store contains {loaded_vectorstore.index.ntotal} vectors")

Loaded vector store contains 4 vectors


In [17]:
## Similarity Search 
query = "Explain deep learning"

result = loaded_vectorstore.similarity_search(query, k=3)
print(len(result))
print(result)

3
[Document(id='2b8e81c5-90e6-4bcb-b66a-cf1bca1a4687', metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolutionized computer vision, NLP, and speech recognition.'), Document(id='f576cae4-f85d-43e2-94ea-749ed3927e6f', metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.'), Document(id='249530eb-cd82-41e5-a847-71e2a77bc0f5', metadata={'source': 'NLP Overview', 'page': 1, 'topic': 'NLP'}, page_content='Natural Language Processing (NLP) is a branch of AI that helps computers understand human la

In [18]:
print("\nRetrieved Documents:")
print(f"Query: {query}\n")
for i, doc in enumerate(result):
    print(f"Document {i+1}:")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}\n")


Retrieved Documents:
Query: Explain deep learning

Document 1:
Content: Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses multiple layers to progressively extract higher-level features from raw input.
        Deep learning has revolutionized computer vision, NLP, and speech recognition.
Metadata: {'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}

Document 2:
Content: Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised, unsupervised, and reinforcement learning.
Metadata: {'source': 'ML Basics', 'page': 1, 'topic': 'ML'}

Document 3:
Content: Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
        It combines computational linguistics with machine learning and deep learning models.
        Applications include chatbots, translation, sent

In [19]:
#   Similarity Search with score

results_with_score = loaded_vectorstore.similarity_search_with_score(query, k=3)
print("\nRetrieved Documents with Similarity Scores:")
for i, (doc, score) in enumerate(results_with_score):
    print(f"Document {i+1}:")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")
    print(f"Similarity Score: {score}\n")


Retrieved Documents with Similarity Scores:
Document 1:
Content: Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses multiple layers to progressively extract higher-level features from raw input.
        Deep learning has revolutionized computer vision, NLP, and speech recognition.
Metadata: {'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}
Similarity Score: 0.669890284538269

Document 2:
Content: Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised, unsupervised, and reinforcement learning.
Metadata: {'source': 'ML Basics', 'page': 1, 'topic': 'ML'}
Similarity Score: 1.2500925064086914

Document 3:
Content: Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
        It combines computational linguistics with machine learning and deep learnin

In [20]:
### Search with metadata filtering
filter_criteria = {"topic": "DL"}
filtered_results = loaded_vectorstore.similarity_search_with_score(
    query,
    k=3,
    filter=filter_criteria
)

print("\nFiltered Retrieved Documents with Similarity Scores:")
for i, (doc, score) in enumerate(filtered_results):
    print(f"Document {i+1}:")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")
    print(f"Similarity Score: {score}\n")


Filtered Retrieved Documents with Similarity Scores:
Document 1:
Content: Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses multiple layers to progressively extract higher-level features from raw input.
        Deep learning has revolutionized computer vision, NLP, and speech recognition.
Metadata: {'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}
Similarity Score: 0.669890284538269



In [21]:
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [22]:
### Build RAG Chain With LCEL 

from langchain.chat_models import init_chat_model
llm = init_chat_model(model="groq:llama-3.1-8b-instant")
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x7a5f0dfc2fc0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7a5f0e0328a0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [23]:
llm.invoke("Hello, who are you?")

AIMessage(content="I'm an artificial intelligence model known as a large language model (LLM) or a conversational AI. I'm a computer program designed to understand and respond to human language in a way that's natural and helpful.\n\nI don't have a personal identity or emotions, but I'm here to assist you with any questions, topics, or tasks you'd like to discuss. I can provide information on a wide range of subjects, from science and history to entertainment and culture.\n\nI've been trained on a massive dataset of text from the internet, books, and other sources, which allows me to generate human-like responses to your queries. I can also engage in conversations, tell stories, and even create text based on your prompts.\n\nSo, what would you like to talk about?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 157, 'prompt_tokens': 41, 'total_tokens': 198, 'completion_time': 0.226127318, 'completion_tokens_details': None, 'prompt_time': 0.002672563, 'pro

In [24]:
# 1. Simple RAG Chain with LCEL
simple_prompt = ChatPromptTemplate.from_template("""Answer the question based only on the following context:
Context: {context}

Question: {question}

Answer:""")

In [25]:
## Basic retriever

retriever = loaded_vectorstore.as_retriever(
    search_type="similarity", 
    search_kwargs={"k":3}
)

In [26]:
retriever.invoke("What is deep learning?")

[Document(id='2b8e81c5-90e6-4bcb-b66a-cf1bca1a4687', metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolutionized computer vision, NLP, and speech recognition.'),
 Document(id='f576cae4-f85d-43e2-94ea-749ed3927e6f', metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.'),
 Document(id='249530eb-cd82-41e5-a847-71e2a77bc0f5', metadata={'source': 'NLP Overview', 'page': 1, 'topic': 'NLP'}, page_content='Natural Language Processing (NLP) is a branch of AI that helps computers understand human la

In [27]:
from typing import List

# Format documents for the prompt
def format_docs(docs: List[Document]) -> str:
    """Format documents for insertion into prompt"""
    formatted = []
    for i, doc in enumerate(docs):
        source = doc.metadata.get('source', 'Unknown')
        formatted.append(f"Document {i+1} (Source: {source}):\n{doc.page_content}")
    return "\n\n".join(formatted)

In [28]:

simple_rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | simple_prompt
    | llm
    | StrOutputParser()
)

simple_rag_chain

{
  context: VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7a5f0e2eb620>, search_kwargs={'k': 3})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\nContext: {context}\n\nQuestion: {question}\n\nAnswer:'), additional_kwargs={})])
| ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x7a5f0dfc2fc0>, async_clien

In [29]:
simple_rag_chain.invoke("Explain deep learning in simple terms.")

"Deep learning is a type of machine learning that uses artificial neural networks to get better at tasks over time. It's like how our brains work, with layers of processing that help us understand more complex things. In deep learning, these layers are used to analyze raw input, like images or sounds, and extract higher-level features that help computers recognize patterns and make decisions. This has led to huge improvements in areas like computer vision, language understanding, and speech recognition."

In [30]:
### Conversational RAg Chain
conversational_prompt = ChatPromptTemplate.from_messages( [ ("system", "You are a helpful AI assistant. Use the provided context to answer questions."),
    ("placeholder", "{chat_history}"),
    ("human", "Context: {context}\n\nQuestion: {input}")]
)


In [31]:
def create_conversational_rag():
    """Create a conversational RAG chain with memory"""
    return (
        RunnablePassthrough.assign(
            context=lambda x: format_docs(retriever.invoke(x["input"]))
        )
        |conversational_prompt
        | llm
        | StrOutputParser()
    )

conversational_rag_chain = create_conversational_rag()
conversational_rag_chain

RunnableAssign(mapper={
  context: RunnableLambda(lambda x: format_docs(retriever.invoke(x['input'])))
})
| ChatPromptTemplate(input_variables=['context', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag=

In [32]:
### streaming RAG chain
streaming_rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | simple_prompt
    | llm
)

print("Modern RAG chains created successfully!")
print("Available chains:")
print("- simple_rag_chain: Basic Q&A")
print("- conversational_rag: Maintains conversation history")
print("- streaming_rag_chain: Supports token streaming")

Modern RAG chains created successfully!
Available chains:
- simple_rag_chain: Basic Q&A
- conversational_rag: Maintains conversation history
- streaming_rag_chain: Supports token streaming


In [33]:
# Test function for different chain types
def test_rag_chains(question: str):
    """Test all RAG chain variants"""
    print(f"Question: {question}")
    print("=" * 80)

    # 1. Simple RAG
    print("\n1. Simple RAG Chain:")
    answer = simple_rag_chain.invoke(question)
    print(f"Answer: {answer}")

    print("\n2. Streaming RAG:")
    print("Answer: ", end="", flush=True)

    for chunk in streaming_rag_chain.stream(question):
        print(chunk.content, end="", flush=True)
    
    print()



In [34]:
test_rag_chains("What is the difference between AI and machine learning")

Question: What is the difference between AI and machine learning

1. Simple RAG Chain:
Answer: Based on the provided context, the difference between AI and Machine Learning is that AI is the simulation of human intelligence in machines, which is a broader field of research and development. It encompasses various types of intelligence, including narrow and general AI.

Machine Learning, on the other hand, is a subset of AI that enables systems to learn from data. It focuses on the development of algorithms that can find patterns in data and make predictions or decisions without being explicitly programmed.

In other words, AI is the umbrella term that includes Machine Learning, as well as other areas like robotics, natural language processing, and expert systems. Machine Learning is a specific technique within AI that deals with the development of algorithms that can learn from data.

2. Streaming RAG:
Answer: Based on the context provided, the difference between AI and machine learning

In [35]:
# Test with multiple questions
test_questions = [
    "What is the difference between AI and Machine Learning?",
    "Explain deep learning in simple terms",
    "How does NLP work?"
]

for question in test_questions:
    print("\n" + "=" * 80 + "\n")
    test_rag_chains(question)



Question: What is the difference between AI and Machine Learning?

1. Simple RAG Chain:
Answer: Based on the given context, AI is defined as the simulation of human intelligence in machines (Document 2), while Machine Learning is a subset of AI that enables systems to learn from data (Document 1). 

Therefore, the main difference between AI and Machine Learning is that AI is a broader concept that encompasses various aspects of simulating human intelligence, whereas Machine Learning is a specific subset of AI that focuses on enabling systems to learn from data through patterns and algorithms.

2. Streaming RAG:
Answer: Based on the given context, the main difference between AI and Machine Learning is that AI is the broader concept of simulating human intelligence in machines, while Machine Learning is a subset of AI that enables systems to learn from data without being explicitly programmed.


Question: Explain deep learning in simple terms

1. Simple RAG Chain:
Answer: Deep learning

In [36]:
## Conversational example
print("\n3. Conversational RAG Example:")
chat_history = []

# first question
question1 = "What is deep learning?"
answer1 = conversational_rag_chain.invoke(
    {
        "input": question1, 
        "chat_history": chat_history
    }
)
print(f"Q: {question1}\nA: {answer1}\n")
chat_history.append(HumanMessage(content=question1))
chat_history.append(AIMessage(content=answer1))


3. Conversational RAG Example:
Q: What is deep learning?
A: Based on the provided context from Document 1, Deep Learning is a subset of machine learning based on artificial neural networks that uses multiple layers to progressively extract higher-level features from raw input.



In [37]:
# Follow-up question
question2 = "Can you explain it in simple terms?"
answer2 = conversational_rag_chain.invoke(
    {
        "input": question2, 
        "chat_history": chat_history
    }
)
print(f"Q: {question2}\nA: {answer2}\n")

Q: Can you explain it in simple terms?
A: Let's break it down in simple terms.

Deep Learning is a type of Artificial Intelligence (AI) that helps computers understand and process complex information from things like images, videos, and language. 

Imagine you're trying to recognize a picture of a cat. A computer would look at the picture and see the following:

- The pixels (small squares of color)
- The shapes (like the cat's ears and tail)
- The colors (like the cat's fur and eyes)

Deep Learning is like looking at the picture in layers. First, the computer sees the pixels, then the shapes, and finally the whole cat. This helps the computer understand what it's looking at and make decisions based on that understanding.

In the case of language, Deep Learning helps computers understand the meaning of words, phrases, and sentences, allowing them to perform tasks like language translation and text summarization.

